In [ ]:
%%configure -f
{
    "conf": {
        "spark.jars.packages": "org.apache.spark:spark-sql-kafka-0-10_2.11:2.2.0,harsha2010:magellan:1.0.5-s_2.11,com.esri.geometry:esri-geometry-api:1.2.1,commons-io:commons-io:2.6,org.apache.spark:spark-streaming_2.11:2.2.0,org.apache.spark:spark-sql_2.11:2.2.0",
        "spark.jars.excludes": "org.scala-lang:scala-reflect,org.apache.spark:spark-tags_2.11",
        "spark.dynamicAllocation.enabled": true,
        "spark.shuffle.service.enabled": true,
        "spark.rdd.compress": true,
        "spark.default.parallelism": 2010,
        "spark.sql.shuffle.partitions": 2010
    }
}


In [ ]:
import util.control.Breaks._
import org.apache.spark.sql.streaming.StreamingQueryListener
import org.apache.spark.util.random.XORShiftRandom
import org.apache.spark.sql.functions.col
import org.apache.spark.sql.types._
import org.apache.spark.rdd.RDD
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.SQLImplicits
import org.apache.spark.sql.functions.from_json
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.ForeachWriter
import magellan._
import magellan.index.ZOrderCurve
import magellan.{Point, Polygon}

import org.apache.spark.sql.magellan.dsl.expressions._
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.streaming.OutputMode
import org.apache.spark.sql.types.{
  DoubleType,
  StringType,
  StructField,
  StructType
}
import org.apache.spark.sql.streaming._
import org.apache.spark.sql.streaming.Trigger
import org.apache.spark.sql.execution.streaming.MemoryStream
import org.apache.spark.sql.functions.{collect_list, collect_set}
import org.apache.spark.sql.SQLContext
import org.apache.log4j.{Level, Logger}
import scala.collection.mutable
import scala.concurrent.duration.Duration
import java.io.{BufferedWriter, FileWriter}
import org.apache.commons.io.FileUtils
import java.io.File
import scala.collection.mutable.ListBuffer
import java.time.Instant
import org.apache.spark.util.CollectionAccumulator
import org.apache.spark.sql.DataFrame

In [ ]:
val containerStorageName = "denis-2021-04-27t09-50-33-745z"
val storageAccountName = "denishdistorage"

In [ ]:
/////////////////////////////
/// Definition of schemas ///
/////////////////////////////

val aerosolDataSchema = StructType(Array(
    StructField("Latitude", DoubleType, false),
    StructField("Longitude", DoubleType, false),
    StructField("Value", DoubleType, false),
    StructField("dataDate", StringType, false),
    StructField("time", StringType, false),
    StructField("shortName", StringType, false)))

val mobilityDataSchema = StructType(Array(
    StructField("Code", StringType, false),
    StructField("Timestamp", StringType, false),
    StructField("Value", DoubleType, false),
    StructField("Latitude", DoubleType, false),
    StructField("Longitude", DoubleType, false),
    StructField("Status", StringType, false),
    StructField("Other_Value", DoubleType, false),
    StructField("Other_Date", StringType, false)))

val pm10DataSchema = StructType(Array(
    StructField("lat", DoubleType, false),
    StructField("lon", DoubleType, false),
    StructField("pm10_value", DoubleType, false)))


In [ ]:
///////////////////////////////
/// Mobility data filenames ///
///////////////////////////////

val mob20k = "db20mila.csv"
val mob100k = "db100mila.csv"
val mob250k = "db250mila.csv"
val mob500k = "db500mila.csv"

In [ ]:
/////////////////////////////
///// Import Dataframes /////
/////////////////////////////

//"wasb[s]://<BlobStorageContainerName>@<StorageAccountName>.blob.core.windows.net/<path>"
val aerosolData = (spark.read.format("csv")
                   .option("header", "true")
                   .schema(aerosolDataSchema)
                   .csv("wasbs://" + containerStorageName + "@" + storageAccountName + ".blob.core.windows.net/data/cams_air_data/cams_data_italy_right_dates.csv")
                   .withColumn("Timestamp", to_timestamp(concat($"dataDate", lit(" "), $"time"), "yyyyMMdd HHmm"))
                   .withColumn("Point", point($"Longitude",$"Latitude"))
                   .drop("dataDate", "time")
                   .repartition(100))

val mobilityData = (spark.read.format("csv")
                    .option("header", "true")
                    .option("delimiter", ";")
                    .schema(mobilityDataSchema)
                    .csv("wasbs://" + containerStorageName + "@" + storageAccountName + f".blob.core.windows.net/data/mobility_data/$mob500k")
                    .withColumn("Timestamp", to_timestamp($"Timestamp", "yyyy-MM-dd HH:mm:ss.SSS"))
                    .withColumn("Point", point($"Longitude",$"Latitude"))
                    .drop("Status", "Other_Value", "Other_Date")
                    .repartition(500))

val pm10Data = (spark.read.format("csv")
                .option("header", "true")
                .schema(pm10DataSchema)
                .csv("wasbs://" + containerStorageName + "@" + storageAccountName + ".blob.core.windows.net/data/copernicus_air_data/pm10.csv")
                .withColumn("Point", point($"lon",$"lat"))
                .repartition(200))

// val minDate = mobilityData.select($"Timestamp").where($"Timestamp".isNotNull).orderBy(asc("Timestamp")).first().mkString(",")
// val maxDate = mobilityData.select($"Timestamp").orderBy(desc("Timestamp")).first().mkString(",")
// val datesString = f"""Min date: $minDate
// Max date: $maxDate"""
// print(datesString)

In [ ]:
// mobilityData.count()

In [ ]:
/////////////////////////////////////////////
/// Boundary coordinates for moblity data ///
/////////////////////////////////////////////

// println("Longitude: " + mobilityData.agg(min("Longitude"), max("Longitude")).head.toString() + "\nLatitude: " + mobilityData.agg(min("Latitude"), max("Latitude")).head.toString())

In [ ]:
//////////////////
/// Geohashing ///
//////////////////

// a user defined function to get geohash from long/lat point
val geohashUDF = udf{(curve: Seq[ZOrderCurve]) => curve.map(_.toBase32())}

val precision: Int = 30

In [ ]:
////////////////////////////
/// Geohash aerosol data ///
////////////////////////////

val geohashedAerosolData = (aerosolData
                            .withColumn("Index", $"Point" index  precision)
                            .withColumn)("GeohashArray1", geohashUDF($"Index.curve"))
val explodedGeohashedAerosolData = (geohashedAerosolData
                                    .explode("GeohashArray1", "Geohash")
                                    { a: mutable.WrappedArray[String] => a })

var explodedGeohashedAerosolDataLight = explodedGeohashedAerosolData.select("Value", "shortName", "Point", "Geohash")

// Renaming of DF columns with prefix for identifying columns in later join
for (column <- explodedGeohashedAerosolDataLight.columns) {
    explodedGeohashedAerosolDataLight = (explodedGeohashedAerosolDataLight
                                    .withColumn("aerosol_" + column, explodedGeohashedAerosolDataLight(column))
                                    .drop(column))
}

In [ ]:
/////////////////////////////
/// Geohash mobility data ///
/////////////////////////////


val geohashedMobilityData = (mobilityData
                         .withColumn("Index", $"Point" index  precision)
                         .withColumn("GeohashArray1", geohashUDF($"Index.curve")))
val explodedGeohashedMobilityData = (geohashedMobilityData
                                 .explode("GeohashArray1", "Geohash")
                                 { a: mutable.WrappedArray[String] => a })
var explodedGeohashedMobilityDataLight = (explodedGeohashedMobilityData
                                          .select("Code", "Value", "Point", "Geohash"))

// Renaming of DF columns with prefix for identifying columns in later join
for (column <- explodedGeohashedMobilityDataLight.columns) {
    explodedGeohashedMobilityDataLight = (explodedGeohashedMobilityDataLight
                                    .withColumn("mobility_" + column, explodedGeohashedMobilityDataLight(column))
                                    .drop(column))
}


In [ ]:
/////////////////////////
/// Geohash pm10 data ///
/////////////////////////

val geohashedPm10Data = (pm10Data
                         .withColumn("Index", $"Point" index  precision)
                         .withColumn("GeohashArray1", geohashUDF($"Index.curve")))
val explodedGeohashedPm10Data = (geohashedPm10Data
                                 .explode("GeohashArray1", "Geohash")
                                 { a: mutable.WrappedArray[String] => a })
var explodedGeohashedPm10DataLight = (explodedGeohashedPm10Data
                                      .select($"lat".as("Latitude"), $"lon".as("Longitude"),
                                              $"pm10_value".as("Value"), $"Point", $"Geohash"))

// Renaming of DF columns with prefix for identifying columns in later join
for (column <- explodedGeohashedPm10DataLight.columns) {
    explodedGeohashedPm10DataLight = (explodedGeohashedPm10DataLight
                                 .withColumn("pm10_" + column, explodedGeohashedPm10DataLight(column))
                                 .drop(column))
}


In [ ]:
/////////////////////////////////////////////////
/// Import and Geohash the polygon of Bologna ///
/////////////////////////////////////////////////

// The final schema is unified as:
// root
//  |-- polygon: polygon (nullable = true)
//  |-- index: array (nullable = false)
//  |    |-- element: struct (containsNull = true)
//  |    |    |-- curve: zordercurve (nullable = false)
//  |    |    |-- relation: string (nullable = false)
//  |-- Neighborhood: string (nullable = true)
//  |-- Province: string (nullable = true)
//  |-- Region: string (nullable = true)
//  |-- geohashArray: array (nullable = true)
//  |    |-- element: string (containsNull = true)
//  |-- geohash: string (nullable = true)

val rawBologna = (spark.read.format("magellan")
                  .option("type", "geojson")
                  .load("wasbs://" + containerStorageName + "@" + storageAccountName + ".blob.core.windows.net/data/geojson/Bologna_quartieri.geojson")
                  .select($"polygon", $"metadata"("NOMEQUART").as("Neighboorhood"))
                  )

val bologna = (rawBologna
               .withColumn("index", $"polygon" index  precision)
               .withColumn("Province", lit("Bologna"))
               .withColumn("Region", lit("Emilia-Romagna"))
               .select($"polygon", $"index", $"Neighboorhood", $"Province", $"Region"))

val zorderIndexedBologna = (bologna
                            .withColumn("index", explode($"index"))
                            .select("polygon", "index.curve", "index.relation","Neighboorhood", "Province", "Region")
                          )
val geohashedBologna = bologna.withColumn("geohashArray", geohashUDF($"index.curve"))
val explodedGeohashedBologna = geohashedBologna.explode("geohashArray", "geohash") { a: mutable.WrappedArray[String] => a }

In [ ]:
///////////////////////////////////////////////
/// Import and Geohash the polygon of Italy ///
///////////////////////////////////////////////

// The final schema is unified as:
// root
//  |-- polygon: polygon (nullable = true)
//  |-- index: array (nullable = false)
//  |    |-- element: struct (containsNull = true)
//  |    |    |-- curve: zordercurve (nullable = false)
//  |    |    |-- relation: string (nullable = false)
//  |-- Neighborhood: string (nullable = true)
//  |-- Province: string (nullable = true)
//  |-- Region: string (nullable = true)
//  |-- geohashArray: array (nullable = true)
//  |    |-- element: string (containsNull = true)
//  |-- geohash: string (nullable = true)


val rawItaly = (spark.read.format("magellan")
                  .option("type", "geojson")
                  .load("wasbs://" + containerStorageName + "@" + storageAccountName + ".blob.core.windows.net/data/geojson/Italy_quartieri.geojson")
                  .select($"Polygon",
                          $"metadata"("name").as("Neighborhood"),
                          $"metadata"("prov_name").as("Province"),
                          $"metadata"("reg_name").as("Region"))
                  )
val italy = (rawItaly
               .withColumn("Index", $"polygon" index  precision)
               .select($"Polygon", $"Index", $"Neighborhood", $"Province", $"Region"))

val zorderIndexedItaly = (italy
                            .withColumn("Index", explode($"Index"))
                            .select("Polygon", "Index.curve", "Index.relation", "Neighborhood", "Province", "Region")
                          )
val geohashedItaly = italy.withColumn("GeohashArray", geohashUDF($"Index.curve"))
val explodedGeohashedItaly = geohashedItaly.explode("GeohashArray", "Geohash") { a: mutable.WrappedArray[String] => a }

In [ ]:
val explodedGeohashedItalyNoBologna = explodedGeohashedItaly.filter(col("Neighborhood") =!= "Bologna")

In [ ]:
var allCities = explodedGeohashedItalyNoBologna.union(explodedGeohashedBologna).repartition(1000)

for (column <- allCities.columns) {
    allCities = (allCities
                 .withColumn("cities_" + column, allCities(column))
                 .drop(column))
}


In [ ]:
val meteoMobilityIntegratedDataframe = (allCities
                                        .join(explodedGeohashedMobilityDataLight,
                                               $"mobility_Geohash" === $"cities_Geohash")
                                        .join(
                                            explodedGeohashedPm10DataLight, 
                                            $"pm10_Geohash" === $"cities_Geohash")
                                        .where($"pm10_Point" within $"cities_Polygon")
                                        .select($"cities_Polygon".as("Polygon"), $"pm10_Longitude".as("Longitude"),
                                                $"pm10_Latitude".as("Latitude"), $"pm10_Point".as("Point"),
                                                $"cities_Neighborhood".as("Neighborhood"),
                                                $"cities_Province".as("Province"), $"cities_Region".as("Region"),
                                                $"cities_Geohash".as("Geohash"), $"mobility_Code".as("Trip_Code"),
                                                $"mobility_Value".as("Trip_Value"), $"pm10_Value".as("PM10_Value"))
                                        .cache()
                                       )

// Saving the output of SparkSession.time() output is unreliable, as the time elapsed is only printed to stdout,
// meanwhile what is returned is the output of the function, as shown in the SparkSession source code:
// 
/**
   * Executes some code block and prints to stdout the time taken to execute the block. This is
   * available in Scala only and is used primarily for interactive testing and debugging.
   *
   * @since 2.1.0
   */
// def time[T](f: => T): T = {
//     val start = System.nanoTime()
//     val ret = f
//     val end = System.nanoTime()
//     // scalastyle:off println
//     println(s"Time taken: ${NANOSECONDS.toMillis(end - start)} ms")
//     // scalastyle:on println
//     ret
//   }

var linesWritten = spark.time(meteoMobilityIntegratedDataframe.count())


In [ ]:
val timeTaken = 81305d
val baseThroughput = linesWritten/timeTaken*1000d
println("Lines written: "+ linesWritten + " lines\nTime elapsed: " + timeTaken + 
        " ms\nThroughput: " + baseThroughput + " lines/s")


In [ ]:
// ///////////////////////////
// /// Top K spatial query ///
// ///////////////////////////


spark.time(meteoMobilityIntegratedDataframe
 .select("*")
 .groupBy(col("Neighborhood"))
 .agg(count("*").as("count"))
 .sort(desc("count"))
 .show(1))

In [ ]:
/////////////////////
/// Average query ///
/////////////////////

spark.time(meteoMobilityIntegratedDataframe.groupBy($"Neighborhood").agg(avg($"PM10_Value")).show(1))